In [ ]:
!python -m spacy download fr_core_news_sm
!pip install simpletransformers

     |████████████████████████████████| 14.7 MB 6.5 MB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-py3-none-any.whl size=14727026 sha256=415a6633c748b980855bc54f23a15a4c76cff0a5694708728a2c82ce3b8879c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-o8vfv8f3/wheels/c9/a6/ea/0778337c34660027ee67ef3a91fb9d3600b76777a912ea1c24
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
     |████████████████████████████████| 221 kB 7.9 MB/s 
     |████████████████████████████████| 2.6 MB 60.5 MB/s 
     |████████████████████████████████| 1.8 MB 45.7 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 264 kB 66.8 MB/s 
     |████████████████████████████████| 1.2 MB 54.6 MB/s 
     |████████████████████████████████| 8.0 MB 64.2 MB/s 
     |████████████████████████████████| 124 kB 68.2 MB/s 
     |████████████████████████████████| 76 kB 6.5

In [ ]:
import ipykernel
ipykernel.version_info

(5, 5, 5)

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
chemincorpus = "/content/drive/MyDrive/Stage Defense/Corpus/"
cheminw2v = chemincorpus + 'W2V_mentions_Defense.csv'
cheminbow = chemincorpus + 'BoW_mentions_Defense.csv'
cheminfeatures = chemincorpus +"features_contexte_Defense.csv"
models_path = "/content/drive/MyDrive/Stage Defense/Scripts/Modeles/"


In [ ]:

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.svm import LinearSVC,SVC
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import shuffle

import torch


In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

2021-08-07 15:05:42.014640: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
import spacy
nlp = spacy.load('fr_core_news_sm')

Fonctions utiles

In [ ]:
def equilibre_classes(X,y,threshold=None):
  '''Retire aléatoirement des exemples des classes surnnuméraires, à la fois dans X et dans y
  Renvoie X et y avec toutes les classes ayant le meme nombre d'exemples que la classe minoritaire'''
  dicoclasses = {k:v for k,v in zip(*np.unique(y, return_counts=True))}
  print("avant",dicoclasses)
  mini = min(dicoclasses.values())
  if threshold:
    mini = threshold
  while any((v >mini for v in dicoclasses.values())):
    
    for cat in dicoclasses:
      
      if dicoclasses[cat] > mini:
        nb_hasard = np.random.choice(np.where(y==cat)[0],1)
        X = np.delete(X,nb_hasard, axis=0)
        y = np.delete(y,nb_hasard, axis=0)
        dicoclasses[cat]-=1

  print("apres",dicoclasses)
  return X,y
  #for i in range(len(X)):


In [ ]:
def conversion_classes_num(y,convert,reverse_conv=False):
  if reverse_conv:
    convert = {v:k for k,v in convert.items()}
  for i in range(len(y)):
    y[i] = convert[y[i]]
  return y

In [ ]:
def validation_croisee_camembert(X,y,convert,n_epochs,n_splits=5, return_model=False,output_dir=None):
  if n_splits > 1:
    kf = StratifiedKFold(n_splits=n_splits, random_state=None, shuffle=True)
    splits = kf.split(X,y)
  else:
    i_train, i_test = train_test_split(list(range(len(X))), test_size=0.1)
    splits = (i_train,i_test)
  y_pred,y_true = [], []
  #print(len(splits), splits)
  for train_index, test_index in splits:
    #print("TRAIN:", train_index, "TEST:", test_index)
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    y_train, y_test = conversion_classes_num(y_train,convert),conversion_classes_num(y_test,convert)
    df_train = pd.DataFrame(zip(X_train,y_train), columns = ["text","labels"])
    df_test = pd.DataFrame(zip(X_test,y_test), columns = ["text","labels"])
    #print(df_train)
    print(convert.values(), np.unique(y_train))
    weights = compute_class_weight('balanced',list(convert.values()),y_train)
    model_args = ClassificationArgs(num_train_epochs=n_epochs,overwrite_output_dir=True,output_dir=output_dir)
    model = ClassificationModel(
      'camembert',
      'camembert-base',
      weight=weights.tolist(),
      num_labels = len(convert),
      args=model_args) 
    
    model.train_model(df_train)
    predictions, raw_outputs = model.predict(X_test.tolist())
    y_pred.extend(predictions)
    y_true.extend(y_test)
  if return_model:
    return y_pred,y_true,model
  return y_pred,y_true

In [ ]:
def liste_lemmes_predicat(phrase,lemme=True,
                          pos_filtre=['VERB','ADJ','ADV','INTJ','NOUN'],
                          lemme_filtre = ['ville','défense']):
    doc = nlp(phrase)
    liste_lemmes = []
    for tok in doc:
      if tok.text == 'ville':
        tete = tok.head
        if tete.pos_ != 'VERB' or len(list(tete.subtree))<3:
          tete = tete.head
        #print(tete,tete.pos_, list(tete.subtree), phrase, sep=' | ')
        for elem in tete.subtree:
          if elem.pos_ in pos_filtre and elem.lemma_  not in lemme_filtre:
            if lemme:
              liste_lemmes.append(elem.lemma_.lower())
            else:
              liste_lemmes.append(elem.text)
    return liste_lemmes

Chargement des données

In [ ]:
dfbow = pd.read_csv(cheminbow, sep=",",index_col=0)
dfw2v = pd.read_csv(cheminw2v, sep=",",index_col=0)
dffeatures = pd.read_csv(cheminfeatures, sep=",",index_col=0)

In [ ]:
chemindata = chemincorpus+"Toutes_mentions_Defense_annotees.csv"

In [ ]:
dfannot = pd.read_csv(chemindata,sep=",", index_col=0, keep_default_na=False,na_values=["NA"])
annotations = dfannot.loc[:,:"positionnement_contexte"]
annotations = annotations.fillna("NA")

In [ ]:
#Régularisation des étiquettes
annotations = annotations.loc[dfbow.index,:]
annotations.cible_contexte = annotations.cible_contexte.str.strip()
annotations.loc[annotations[annotations.cible_contexte=="quotidien"].index,'cible_contexte'] = "vie quotidienne"

# Predictions Contexte

## Cible Contexte

### CamemBERT

Na Vs Non-NA

In [ ]:

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
X_contexte = dffeatures.contexte.to_numpy()
y_cible_na = np.where(y_cible_c != 'NA', "non_na", y_cible_c)


In [ ]:
n_splits = 5
n_epochs = 15

In [ ]:
models_path = "/content/drive/MyDrive/Stage Defense/Scripts/Modeles/"
NA_model_path = models_path+ "NA_non_NA_Model"

In [ ]:
convert = {'NA':0,"non_na":1}
y_pred,y_true,model = validation_croisee_camembert(X_contexte,y_cible_na,convert,n_epochs,n_splits,return_model=True,output_dir=NA_model_path)

dict_values([0, 1]) [0 1]


INFO:filelock:Lock 140131878914192 acquired on /root/.cache/huggingface/transformers/f459e43c5ebb871abbf9209195563bff6a11547fd9532047739667c394833221.e23d229c54bcc6f67d337b8b2dd111b0e3dc01fa854bfecd3efdeb8c955749e6.lock


Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

INFO:filelock:Lock 140131878914192 released on /root/.cache/huggingface/transformers/f459e43c5ebb871abbf9209195563bff6a11547fd9532047739667c394833221.e23d229c54bcc6f67d337b8b2dd111b0e3dc01fa854bfecd3efdeb8c955749e6.lock
INFO:filelock:Lock 140131878914192 acquired on /root/.cache/huggingface/transformers/7e23f45751ad1fed420ca9f03bb37a279dc98a56c75bf25e671129237e2c893c.ee4d4253e08a7cf9697c0671fd8f022483dbf586691a7b32ead55493a34d72b2.lock


Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

...

  0%|          | 0/166 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

dict_values([0, 1]) [0 1]


Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'clas

KeyboardInterrupt: ignored

In [ ]:
y_true =  conversion_classes_num(y_true,convert,reverse_conv=True)
y_pred =  conversion_classes_num(y_pred,convert,reverse_conv=True)
report = classification_report(y_true,y_pred)
print(report)

              precision    recall  f1-score   support

          NA       0.74      0.54      0.62       247
      non_na       0.82      0.92      0.87       580

    accuracy                           0.81       827
   macro avg       0.78      0.73      0.75       827
weighted avg       0.80      0.81      0.80       827



### Extraction des NA

In [ ]:
index_na = annotations[annotations.cible_contexte == 'NA'].index
index_non_na = annotations[annotations.cible_contexte != 'NA'].index

### Autres cibles

In [ ]:
def reequilibre_classes(X,y,cible_support):
  
  classes,counts = np.unique(np.array([label  for labels in y for label in labels]),return_counts=True)
  dico_counts = {classe: count for classe,count in zip(classes,counts)}
  dico_counts = dict(sorted(dico_counts.items(),key=lambda x:x[1] , reverse=True))
  print("avant", dico_counts)
  #print(X)

  new_X,new_y = [], []
  if not isinstance(X,list):
    X= X.tolist()
  nb_sup,nb_add = 0,0
  nb_total = len(X)
  for label in dico_counts:
    print(label,"avant", dico_counts)
    while dico_counts[label]  < cible_support   or dico_counts[label]  > cible_support:
      i = random.randint(0,len(y)-1)
      labels = y[i]
      if label not in labels : continue
      if dico_counts[label] < cible_support :
        for lab in labels:
          dico_counts[lab] +=1
        X.append(X[i])
        y.append(y[i])
        nb_add +=1
      elif dico_counts[label] > cible_support :
        if any((dico_counts[lab] < cible_support-5 for lab in labels) ): continue
        for lab in labels:
          dico_counts[lab] -=1
        del X[i]
        del y[i]
        nb_sup+=1
    
    print(label,"après", dico_counts)

  classes,counts = np.unique(np.array([label  for labels in y for label in labels]),return_counts=True)
  dico_counts = {classe: count for classe,count in zip(classes,counts)}
  print(f"exemples avant : {nb_total}", f"ajoutés : {nb_add}", f"supprimés : {nb_sup}", f"exemples après : {len(X)}", sep = " | ")


  new_dico_counts =   {classe: count for classe,count in zip(*np.unique(np.array([label  for labels in new_y for label in labels]),return_counts=True))}
  print("a la fin",new_dico_counts)
  return X,y

### CamemBERT

In [ ]:
X_cible = dffeatures.loc[index_non_na,'contexte'].to_numpy()
y_cible = annotations.loc[index_non_na,'cible_contexte'].to_numpy()

In [ ]:
n_splits = 10
n_epochs = 15
convert = {'économie':0, 'vie quotidienne':1, 'urbanisme':2, 'événement':3,
       'administration':4, 'tourisme':5,  'immobilier':6,}
#y_cible = conversion_classes_num(y_cible,convert)

In [ ]:
np.unique(y_cible)

array(['administration', 'immobilier', 'tourisme', 'urbanisme',
       'vie quotidienne', 'économie', 'événement'], dtype=object)

In [ ]:
y_pred, y_true, model_cible_contexte = validation_croisee_camembert(X_cible,y_cible,convert,n_epochs,n_splits,return_model=True,output_dir=models_path+ "cible_contexte_Model")

dict_values([0, 1, 2, 3, 4, 5, 6]) [0 1 2 3 4 5 6]


Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'clas

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/66 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:942: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm


...

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'clas

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/66 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/66 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
y_true = conversion_classes_num(y_true,convert,reverse_conv=True)
y_pred = conversion_classes_num(y_pred,convert,reverse_conv=True)
report = classification_report(y_true,y_pred)

In [ ]:
print(report)

                 precision    recall  f1-score   support

 administration       0.77      0.72      0.74        68
     immobilier       0.52      0.56      0.54        45
       tourisme       0.56      0.58      0.57        31
      urbanisme       0.72      0.76      0.73       143
vie quotidienne       0.75      0.77      0.76       142
       économie       0.81      0.76      0.79        84
      événement       0.78      0.69      0.73        67

       accuracy                           0.72       580
      macro avg       0.70      0.69      0.69       580
   weighted avg       0.73      0.72      0.72       580



## Temporalité

In [ ]:
n_splits = 10
n_epochs = 15
convert = {'passé':0, 'présent':1, 'futur':2,}

In [ ]:
y_pred,y_true, model_tempo = validation_croisee_camembert(X_phrase,y_tempo,convert,n_epochs, n_splits, return_model=True,output_dir = models_path+ "temporalité_Model")



dict_values([0, 1, 2]) [0 1 2]


Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'clas

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/66 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:942: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm


...

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'clas

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/66 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/66 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
y_true = conversion_classes_num(y_true,convert,reverse_conv=True)
y_pred = conversion_classes_num(y_pred,convert,reverse_conv=True)
report = classification_report(y_true,y_pred)

In [ ]:
print(report)

              precision    recall  f1-score   support

       futur       0.65      0.65      0.65       127
       passé       0.64      0.51      0.57        57
     présent       0.82      0.85      0.84       396

    accuracy                           0.77       580
   macro avg       0.70      0.67      0.68       580
weighted avg       0.77      0.77      0.77       580



## Polarité

In [ ]:
X_phrase = dffeatures.loc[index_non_na,"contexte"].to_numpy()
y_pola_c = annotations.loc[index_non_na,'polarité_contexte']

In [ ]:
n_splits = 5
n_epochs = 15
convert = {'négatif':0, 'neutre':1, 'positif':2}

In [ ]:
y_pred,y_true, model_pola = validation_croisee_camembert(X_phrase,y_pola_c.to_numpy(),convert,n_epochs, n_splits,return_model=True,output_dir = models_path+ "polarité_contexte_Model")

dict_values([0, 1, 2]) [0 1 2]


Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'clas

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/66 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/66 [00:00<?, ?it/s]

...

Running Epoch 1 of 15:   0%|          | 0/66 [00:00<?, ?it/s]

Running Epoch 2 of 15:   0%|          | 0/66 [00:00<?, ?it/s]

Running Epoch 3 of 15:   0%|          | 0/66 [00:00<?, ?it/s]

Running Epoch 4 of 15:   0%|          | 0/66 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
y_true = conversion_classes_num(y_true,convert,reverse_conv=True)
y_pred = conversion_classes_num(y_pred,convert,reverse_conv=True)
report = classification_report(y_true,y_pred)

In [ ]:
print(report)

              precision    recall  f1-score   support

      neutre       0.65      0.66      0.66       244
     négatif       0.74      0.59      0.66       120
     positif       0.61      0.68      0.64       216

    accuracy                           0.65       580
   macro avg       0.67      0.64      0.65       580
weighted avg       0.66      0.65      0.65       580



## Positionnement contexte

In [ ]:
n_splits = 5
n_epochs = 15
convert = {'pris en charge':0, 'cité':1, 'attribué':2}

In [ ]:
y_pred,y_true,model_positionnement = validation_croisee_camembert(X_phrase,y_position.to_numpy(),convert,n_epochs, n_splits,return_model=True,output_dir = models_path+ "positionnement_contexte_Model")

dict_values([0, 1, 2]) [0 1 2]


Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'clas

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/66 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/66 [00:00<?, ?it/s]

...

dict_values([0, 1, 2]) [0 1 2]


Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'clas

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/66 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
y_true = conversion_classes_num(y_true,convert,reverse_conv=True)
y_pred = conversion_classes_num(y_pred,convert,reverse_conv=True)
report = classification_report(y_true,y_pred)

In [ ]:
print(report)

                precision    recall  f1-score   support

      attribué       0.82      0.69      0.75        13
          cité       0.83      0.77      0.80       117
pris en charge       0.94      0.96      0.95       450

      accuracy                           0.91       580
     macro avg       0.86      0.81      0.83       580
  weighted avg       0.91      0.91      0.91       580

